# M1. Actividad

## Propósito

Conocer y aplicar una herramienta para la implementación de sistemas multiagentes.

---

## Instrucciones

Para este problema, deberás entregar, **de manera individual**, un informe en un cuaderno de Jupyter (de no usar Jupyter se tendrá que entregar un PDF) que estudie las estadísticas de un robot de limpieza reactivo, así como el enlace al repositorio en Github del código desarrollado para esta actividad. El código debe ajustarse al estilo solicita en el siguiente documento.

Dado:

* Habitación de MxN espacios.
* Número de agentes.
* Porcentaje de celdas inicialmente sucias.
* Tiempo máximo de ejecución.

Realiza la siguiente simulación:

* Inicializa las celdas sucias (ubicaciones aleatorias).
* Todos los agentes empiezan en la celda [1,1].
* En cada paso de tiempo:
    * Si la celda está sucia, entonces aspira.
    * Si la celda está limpia, el agente elije una dirección aleatoria para moverse (unas de las 8 celdas vecinas) y elije la acción de movimiento (si no puede moverse allí, permanecerá en la misma celda).
* Se ejecuta el tiempo máximo establecido.

Deberás recopilar la siguiente información durante la ejecución:
* Tiempo necesario hasta que todas las celdas estén limpias (o se haya llegado al tiempo máximo).
* Porcentaje de celdas limpias después del termino de la simulación.
* Número de movimientos realizados por todos los agentes.

Analiza cómo la cantidad de agentes impacta el tiempo dedicado, así como la cantidad de movimientos realizados. Desarrollar un informe con lo observado.

---

## ¿Bajo qué criterios se evalúa mi evidencia?

45% - El código cumple correctamente con la funcionalidad requerida por parte de la actividad.

10% - El código deberá seguir los lineamientos estipulados en el estándar de codificación: [liga_estándar_codificación](https://experiencia21.tec.mx/courses/182839/files/52846174?wrap=1)

45% - El informe presenta un análisis del problema planteado, así como las conclusiones obtenidas del mismo.

---

## Especificaciones de entrega

Entrega tu reporte individual en Canvas en el botón "Entregar Tarea" en la parte superior de esta pantalla.

---

## Imports

In [1]:
from mesa import Agent, Model 
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd
import random

import time
import datetime

## Crear el modelo

In [29]:
def obtener_habitacion(modelo):
    habitacion = np.zeros((modelo.grid.width, modelo.grid.height))
    for celda in modelo.grid.coord_iter():
        contenido_celda, x, y = celda
        for contenido in contenido_celda:
            if isinstance(contenido, Aspiradora):
                habitacion[x][y] = 2
            else:
                habitacion[x][y] = contenido.estado
    return habitacion
    
class Aspiradora(Agent):
    
    def __init__(self, id_unico, modelo):
        super().__init__(id_unico, modelo)
        self.nueva_posicion = None
        self.movimientos = 0
        
    def step(self):
        vecinos = self.model.grid.get_neighbors(
            self.pos,
            moore=True,
            include_center=True)
        
        # Defino el siguiente estado que va a tener el piso para la siguiente iteracion sin asignarlo todavia eso lo hago en el método `advance`.
        for vecino in vecinos:
            if isinstance(vecino, Piso) and vecino.pos == self.pos:
                vecino.siguiente_estado = vecino.estado
                if vecino.siguiente_estado == vecino.SUCIO: 
                    vecino.siguiente_estado = vecino.LIMPIO
                    self.nueva_posicion = self.pos
                else:
                    vecindario = self.model.grid.get_neighborhood(
                        self.pos,
                        moore=True,
                        include_center=False)
                    nueva_posicion = self.random.choice(vecindario)
                    self.nueva_posicion = nueva_posicion
                break
        
    def advance(self):
        
        # Actualizamos el estado del piso
        vecinos = self.model.grid.get_neighbors(
            self.pos,
            moore=False,
            include_center=True)
        
        for vecino in vecinos:
            if isinstance(vecino, Piso) and vecino.pos == self.pos: 
                vecino.estado = vecino.siguiente_estado
                break
        
        if self.pos != self.nueva_posicion:
            self.movimientos = self.movimientos + 1
            
        # Movemos la aspiradora a su nueva posicion
        self.model.grid.move_agent(self, self.nueva_posicion)
    
class Piso(Agent):
    
    SUCIO = 1
    LIMPIO = 0
    
    def __init__(self, pos, modelo, estado=LIMPIO):
        super().__init__(pos, modelo)
        self.x, self.y = pos
        self.estado = estado
        self.siguiente_estado = None

class Habitacion(Model):
    
    def __init__(self, m, n, num_agentes, por_celdas_sucias):
        self.num_agentes = num_agentes
        self.por_celdas_sucias = por_celdas_sucias
        self.por_celdas_limpias = 1 - por_celdas_sucias
        self.grid = MultiGrid(m, n, True)
        self.schedule = SimultaneousActivation(self)
               
        # Posicionar celdas sucias de forma aleatoria
        celdas_sucias = int((m * n) * por_celdas_sucias)
        lista_celdas_vacias = list(self.grid.empties)
        for celdas in range(celdas_sucias):
            celda_vacia = random.choice(lista_celdas_vacias)
            piso = Piso(celda_vacia, self)
            piso.estado = piso.SUCIO
            self.grid.place_agent(piso, celda_vacia)
            self.schedule.add(piso)
            lista_celdas_vacias.remove(celda_vacia)
        
        # Posicionar celdas limpias
        lista_celdas_vacias = list(self.grid.empties)
        for celdas in lista_celdas_vacias:
            piso = Piso(celdas, self)
            self.grid.place_agent(piso, celdas)
            self.schedule.add(piso)
        
        # Posicionar agentes aspiradoras
        for i in range(num_agentes):
            aspiradora = Aspiradora(i, self)
            self.grid.place_agent(aspiradora, (1,1))
            self.schedule.add(aspiradora)
            
        self.colectordatos = DataCollector(
            model_reporters={'Habitacion': obtener_habitacion},
            agent_reporters={'Movimientos': lambda a: getattr(a, 'movimientos', None)}
        )
    
    def step(self):
        self.colectordatos.collect(self)
        self.schedule.step()    
    
    def todasceldaslimpias(self):
        celdas_limpias = 0
        for celda in self.grid.coord_iter():
            contenido_celda, x, y = celda
            for contenido in contenido_celda:
                if isinstance(contenido, Piso) and contenido.estado == contenido.LIMPIO:
                    celdas_limpias = celdas_limpias + 1
        
        self.por_celdas_limpias = celdas_limpias / (self.grid.width * self.grid.height)
        if self.por_celdas_limpias == 1:
            return True
        else:
            return False

Correr el modelo

In [30]:
# Datos de la habitacion:
M = 15
N = 15

# Numero de agentes
NUM_AGENTES = 3

# Porcentaje de celdas inicialmente sucias:
PORCENTAJE_CELDAS_SUCIAS = 0.5

# Tiempo máximo de ejecución (segundos)
TIEMPO_MAXIMO_EJECUCION = 0.06

start_time = time.time()
tiempo_inicio = str(datetime.timedelta(seconds=TIEMPO_MAXIMO_EJECUCION))
modelo = Habitacion(M, N, NUM_AGENTES, PORCENTAJE_CELDAS_SUCIAS)

while((time.time() - start_time) < TIEMPO_MAXIMO_EJECUCION and not modelo.todasceldaslimpias()):
    modelo.step()
    #print(str(datetime.timedelta(seconds=(time.time() - start_time))))

# Imprimimos el tiempo que le tomó correr al modelo.
tiempo_ejecucion = str(datetime.timedelta(seconds=(time.time() - start_time)))

## Visualización

In [31]:
todas_habitaciones = modelo.colectordatos.get_model_vars_dataframe()

In [32]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(todas_habitaciones.iloc[0][0], cmap='Greys')

def animate(i):
    patch.set_data(todas_habitaciones.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(todas_habitaciones))

In [33]:
anim

## Informe

In [34]:
movimientos = modelo.colectordatos.get_agent_vars_dataframe()

print('Tiempo necesario hasta que todas las celdas estén limpias:', tiempo_ejecucion, '/', tiempo_inicio)
print('Porcentaje de celdas limpias después del termino de la simulación:', modelo.por_celdas_limpias)
print('Número de movimientos realizados por todos los agentes:', movimientos.tail()['Movimientos'].sum())

Tiempo necesario hasta que todas las celdas estén limpias: 0:00:00.060218 / 0:00:00.060000
Porcentaje de celdas limpias después del termino de la simulación: 0.7244444444444444
Número de movimientos realizados por todos los agentes: 215.0
